In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("UNSW_NB15_training-set.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
df.dtypes

In [ ]:
# len(df["proto"].unique())
# df["sload"]

# find state column encodings
# oglabels = df["proto"].unique()

df["attack_cat"].unique()

# visualize the attack cat column

# df["attack_cat"].value_counts().plot(kind="bar")

df


In [ ]:

df["proto"] = le.fit_transform(df["proto"])
df["service"] = le.fit_transform(df["service"])
df["state"] = le.fit_transform(df["state"])
df["attack_cat"] = le.fit_transform(df["attack_cat"])

In [ ]:
# df.dtypes
import json

# df["service"].unique()

encodings_dict = {
    "proto": oglabels.tolist(),
    "proto_encoded": encoded.tolist()
}

result = {

}
    
for i in range(len(oglabels)):
    result[oglabels[i]] = encoded[i]

print(result)

# encoded_labels_json = json.dumps(encodings_dict)

# with open('encoded_labels.json', 'w') as json_file:
#     json_file.write(encodings_dict)

In [ ]:
imp_col = [
    'id',
 'proto',
 'service',
 'state',
 'rate',
 'sttl',
 'dload',
 'swin',
 'stcpb',
 'dtcpb',
 'dwin',
 'ct_state_ttl',
 'attack_cat',
 'label']

In [ ]:
# df.label.unique()
df

In [ ]:
dict = {6:"Normal", 7:"Reconnaissance",1:"Backdoor", 2:"Dos",3:"Exploits",0:"Analysis", 4:"Fuzzers", 9:"Worms", 8:"SHellcode", 5:"Get Request"}

In [ ]:
df.corr()

In [ ]:
df = df[imp_col]

In [ ]:
# df.corr()
# print rows with attack cat 5
df[df["attack_cat"] == 6]


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(df.corr())

In [ ]:
df = df.drop(columns=['id'])

In [ ]:
df = df.drop(columns=['attack_cat'])

In [ ]:
df

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
x = df.iloc[:,:-1]
y = df.iloc[:, -1]

In [ ]:
x

In [ ]:
y

In [ ]:
df_x_train,df_x_test,df_y_train,df_y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
clf.fit(df_x_train,df_y_train)

In [ ]:
df_y_pred = clf.predict(df_x_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy sccore in RandomForestClassifier: ",accuracy_score(df_y_pred,df_y_test))

In [ ]:
import joblib

joblib.dump(clf, 'random_classifier_model.pkl')

In [ ]:
df_y_pred.tolist()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
import io
import os
import requests
from sklearn import metrics

In [ ]:
cols_used = ['proto', 'service', 'state', 'rate', 'sttl', 'dload', 'swin', 'stcpb','dtcpb', 'dwin', 'ct_state_ttl', 'label']

In [ ]:
cols_trained = ['proto', 'service', 'state', 'rate', 'sttl', 'dload', 'swin', 'stcpb','dtcpb', 'dwin', 'ct_state_ttl']

In [ ]:
# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

In [ ]:
df_x_train

In [ ]:
# Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

In [ ]:
import xgboost as xgb


In [ ]:
# Build the neural network
model = Sequential()
# Hidden 1
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu')) 
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(12, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test), verbose=2,epochs=1000)

In [ ]:
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

In [ ]:
x_test[2]

In [ ]:
# pred.tolist()

# # display all unique values in this pred

pred = pd.array(pred.reshape(-1))

pred.unique().value_counts()

In [ ]:
y_test[22]

In [ ]:
from tabgan.sampler import GANGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
!pip install tabgan

In [ ]:
gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, \
              is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100, 
               "learning_rate": 0.02, "random_state": \
                42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,\
           gan_params = {"batch_size": 500, "patience": 25, \
          "epochs" : 500,}).generate_data_pipe(df_x_train, df_y_train,\
          df_x_test, deep_copy=True, only_adversarial=False, \
          use_adversarial=True)

In [ ]:
gen_x

In [ ]:
# Predict
pred = model.predict(gen_x.values)
score = np.sqrt(metrics.mean_squared_error(pred,gen_y.values))
#more the rmse towards 0 better the model
print("Final score (RMSE): {}".format(score))

In [ ]:
model.save("ids2.keras")

In [ ]:


newmod = load_model("ids2.keras")

newmod.summary()

In [ ]:
model.save_weights("ids_weights.h5")

In [ ]:
# from tensorflow.keras.models import load_model

# newmod = load_model("ids.keras")

# pred = newmod.predict(x_test)
# score = np.sqrt(metrics.mean_squared_error(pred,y_test))
# print("Final score (RMSE): {}".format(score))

x_test[510]

In [ ]:
# input =  [
#   111,
#   5,
#   2,
#   2.10349178314209,
#   64,
#   3621.37158203125,
#   369,
#   1845764736,
#   2769450240,
#   369,
#   0
# ]


# input = [
#   111,
#   5,
#   2,
#   1.8418467044830322,
#   64,
#   3159.135498046875,
#   369,
#   2750092032,
#   1906955392,
#   369,
#   0
# ]
input = [
  111,
  5,
  4,
  1.0162601470947266,
  64,
  325.2032470703125,
  65535,
  2249695488,
  0,
  65535,
  0
]

input = np.array(input)

input = input.reshape(1,11)
print(input.shape)
# newmod.predict(x_test[9710].reshape(1,11))
res = model.predict(input)

res


# select last weights

